## Assignment 5 (50 marks)
#### =====================================================================================================
### Deadline: 11/16 11:59 pm
#### =====================================================================================================

### Problem 1: Regression (10 marks)

The dataset `lab05_dataset_1.csv` has the results of fraud investigations of 5,960 cases. For this task, you will use the input features *FRAUD, TOTAL_SPEND, NUM_CLAIMS, MEMBER_DURATION, OPTOM_PRESC, NUM_MEMBERS* to predict the output *DOCTOR_VISITS*.

### 1.a (2 marks)

1. Normalize the entire dataset.
2. Use a train-test split of 80-20.

In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def train_test_scaled(file: str = 'lab05_dataset_1.csv'):
    df = pd.read_csv(file)
    df_scaled = pd.DataFrame(StandardScaler().fit_transform(df), columns=df.columns)
    Xn = df_scaled.drop(columns=['DOCTOR_VISITS'])
    y = df_scaled['DOCTOR_VISITS'] 
    
    return train_test_split(Xn, y, test_size=0.20, random_state=42) 

if __name__ == '__main__':
    train_test_scaled()

### 1.b (4 marks)

Use sklearn's `LinearRegression` and `SGDRegressor` models for training and output their mean squared error on the test set. Your MSE should be less 0.95.

In [19]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error

FEATURES = ['FRAUD', 'TOTAL_SPEND', 'NUM_CLAIMS', 'MEMBER_DURATION', 'OPTOM_PRESC', 'NUM_MEMBERS']

def lin_reg():
    X_train, X_test, y_train, y_test = train_test_scaled()
    reg = LinearRegression().fit(X_train, y_train)
    y_pred = reg.predict(X_test)

    return mean_squared_error(y_test, y_pred)

def sgd_reg():
    X_train, X_test, y_train, y_test = train_test_scaled()

    reg = SGDRegressor(max_iter=2000, tol=1e-3)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)

    return mean_squared_error(y_test, y_pred) 

if __name__ == '__main__':
    mse_lr = lin_reg()
    mse_sgd = sgd_reg()

    print(f"Linear Regression (MSE): {mse_lr:.4f}")
    print(f"SGD Regression (MSE): {mse_sgd:.4f}")


Linear Regression (MSE): 0.9117
SGD Regression (MSE): 0.9113


### 1.c (4 marks)

Use sklearn's `KNeighborsRegressor` model for training and output their mean squared error on the test set. Your MSE should be less 0.91. Feel free to tweak the parameters of neighbors, algorithm, and metric in the `KNeighborsRegressor` function to reach the desired MSE.

### Problem 2: MLP Classification (20 marks)

`lab05_dataset_2.csv` contains 103,904 observations on airlines customer reviews. For this classification task, use the sklearn's `MLPClassifier` method, where the output class is the `Satisfaction` column which has a binary value of `Satisfied` or `Unsatisfied` and the remaining columns are the input features.

### 2.a (5 marks)

1. Drop any rows with missing values.
2. Output the new length of the final training dataset.
3. Apply encoding to convert all categorical features into numerical values.
4. Normalize the input features.
5. Apply a train-test split of 70-30.

### 2.b (10 marks)

Build two `MLPClassifier` neural networks such that `MSE < 0.05` between the test cases and the neural network predictions for both the networks. You are free to use any number of hidden layers, neurons per layer and activation function to reach your goal. However, both the networks must have the same number of hidden layers and neurons per layer, the only thing that should be different in the networks are their activation functions. Output the MSE for both the networks.

### 2.c (5 marks)

Plot the training loss trajectory together for both the neural network models generated above using two different activation functions. Make sure to add legends and labels to your plot.

### Problem 3: MLP Regression (20 marks)

`lab05_dataset_3.csv` contains 973 observations on gym data. For this prediction task, use the sklearn's `MLPRegressor` method, where the real-valued output feature is the `BMI` column and the remaining columns are the input features.

### 3.a (5 marks)

1. Drop any rows with missing values.
2. Output the new length of the final training dataset.
3. Apply encoding to convert all categorical features into numerical values.
4. Normalize the input features.
5. Apply a train-test split of 80-20.

### 3.b (10 marks)

Build two `MLPRegressor` neural networks such that `MSE < 0.02` between the test cases and the neural network predictions for both the networks. You are free to use any number of hidden layers, neurons per layer and activation function to reach your goal. However, both the networks must have the same number of hidden layers and neurons per layer, the only thing that should be different in the networks are their activation functions. Output the MSE for both the networks.

### 3.c (5 marks)

Plot the training loss trajectory together for both the neural network models generated above using two different activation functions. Make sure to add legends and labels to your plot.